In [198]:
import scipy.sparse as sp
from copy import copy
import math
import heapq
import multiprocessing
import numpy as np
from time import time
from tensorflow import keras 
import pickle
# Global vars
_model = None
_testRatings = None
_testNegatives = None
_K = None

In [199]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [200]:
ratings = pd.read_csv("../data/users_ratings.csv")
ratings.head()

,userId,rating,recipeId
0,2046,5,517
1,1773,4,278
2,2046,5,3431
3,2312,5,780
4,2312,4,1232


In [201]:
user_enc = LabelEncoder()
ratings['user'] = user_enc.fit_transform(ratings['userId'].values)
n_users = ratings['user'].nunique()

item_enc = LabelEncoder()
actuals = list(set(ratings['userId'].values.tolist()))
ratings['recipe'] = item_enc.fit_transform(ratings['recipeId'].values)
actual_items = list(set(ratings['recipeId'].values.tolist()))
n_movies = ratings['recipe'].nunique()

ratings['rating'] = ratings['rating'].values.astype(np.float32)
min_rating = min(ratings['rating'])
max_rating = max(ratings['rating'])

X = ratings[['user', 'recipe']].values
y = ratings['rating'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=17)

X_train.shape, X_test.shape, y_train.shape, y_test.shape


((4635, 2), (515, 2), (4635,), (515,))

In [202]:
n_factors = 60
X_train_array = [X_train[:, 0], X_train[:, 1]]
X_test_array = [X_test[:, 0], X_test[:, 1]]

In [203]:
from keras.layers import Add, Activation, Lambda, Input, Embedding
from keras.layers import Reshape, Flatten, Dot
from keras.layers import Concatenate, Dense, Dropout
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential, Model, load_model, save_model
from keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

class EmbeddingLayer:
    def __init__(self, n_items, n_factors):
        ## keras.initializers.Constant(matrix)
        self.n_items = n_items
        self.n_factors = n_factors
    
    def __call__(self, x):
        x = Embedding(self.n_items, self.n_factors, embeddings_initializer='he_normal',
                      embeddings_regularizer=l2(1e-6))(x)
        x = Reshape((self.n_factors,))(x)
        return x
    
def RecommenderV2(n_users, n_movies, n_factors, min_rating, max_rating):
    user = Input(shape=(1,))
    u = EmbeddingLayer(n_users, n_factors)(user)
    ub = EmbeddingLayer(n_users, 1)(user)
    
    movie = Input(shape=(1,))
    m = EmbeddingLayer(n_movies, n_factors)(movie)
    mb = EmbeddingLayer(n_movies, 1)(movie)
    x = Dot(axes=1)([u, m])
    x = Add()([x, ub, mb])
    x = Activation('sigmoid')(x)
    x = Lambda(lambda x: x * (max_rating - min_rating) + min_rating)(x)
    model = Model(inputs=[user, movie], outputs=x)
    opt = Adam(lr=0.001)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model, u, m


In [204]:
model, uemb, memb = RecommenderV2(n_users, n_movies, n_factors, min_rating, max_rating)
model.summary()


Model: "model_37"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_75 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 input_76 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 embedding_94 (Embedding)       (None, 1, 60)        184560      ['input_75[0][0]']               
                                                                                                  
 embedding_96 (Embedding)       (None, 1, 60)        59880       ['input_76[0][0]']               
                                                                                           

/home/bin/anaconda/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [205]:
history = model.fit(x=X_train_array, y=y_train, batch_size=128, epochs=2222, shuffle=True,
                    verbose=1, validation_data=(X_test_array, y_test))

model.save("./cf_short_net_2222ep.h5")

## save users embeddings ##
w1, w2 = None, None
cnt = 0
for layer in model.layers:
    num = str(layer.name)[-2:]
    if isinstance(layer, Embedding):
        cnt += 1
        if cnt == 1:
            w1 = layer
    if isinstance(layer, Embedding):
        cnt += 1
        if cnt == 2:
            w1 = layer
print(np.array(w1.get_weights()).shape)
#(1, 3076, 60)

uembed = np.array(w1.get_weights())
with open("users_embedding.pkl", "wb") as f:
    pickle.dump(uembed, f)

Epoch 1/2222
37/37 [==============================] - 1s 11ms/step - loss: 3.1483 - accuracy: 0.0345 - val_loss: 2.9578 - val_accuracy: 0.0350
Epoch 2/2222
37/37 [==============================] - 0s 5ms/step - loss: 3.0203 - accuracy: 0.0345 - val_loss: 2.8910 - val_accuracy: 0.0350
Epoch 3/2222
37/37 [==============================] - 0s 4ms/step - loss: 2.8857 - accuracy: 0.0345 - val_loss: 2.8263 - val_accuracy: 0.0350
Epoch 4/2222
37/37 [==============================] - 0s 5ms/step - loss: 2.7277 - accuracy: 0.0345 - val_loss: 2.7648 - val_accuracy: 0.0350
Epoch 5/2222
37/37 [==============================] - 0s 5ms/step - loss: 2.5370 - accuracy: 0.0345 - val_loss: 2.7062 - val_accuracy: 0.0350
Epoch 6/2222
37/37 [==============================] - 0s 5ms/step - loss: 2.3117 - accuracy: 0.0345 - val_loss: 2.6506 - val_accuracy: 0.0350
Epoch 7/2222
37/37 [==============================] - 0s 4ms/step - loss: 2.0601 - accuracy: 0.0345 - val_loss: 2.5998 - val_accuracy: 0.0350
Epoch

37/37 [==============================] - 0s 4ms/step - loss: 0.0208 - accuracy: 0.0345 - val_loss: 2.2757 - val_accuracy: 0.0350
Epoch 59/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0202 - accuracy: 0.0345 - val_loss: 2.2757 - val_accuracy: 0.0350
Epoch 60/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0197 - accuracy: 0.0345 - val_loss: 2.2753 - val_accuracy: 0.0350
Epoch 61/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0193 - accuracy: 0.0345 - val_loss: 2.2759 - val_accuracy: 0.0350
Epoch 62/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0189 - accuracy: 0.0345 - val_loss: 2.2753 - val_accuracy: 0.0350
Epoch 63/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0185 - accuracy: 0.0345 - val_loss: 2.2759 - val_accuracy: 0.0350
Epoch 64/2222
37/37 [==============================] - 0s 4ms/step - loss: 0.0182 - accuracy: 0.0345 - val_loss: 2.2758 - val_accuracy: 0.0350
Epoch 65/2222

37/37 [==============================] - 0s 5ms/step - loss: 0.0121 - accuracy: 0.0345 - val_loss: 2.2710 - val_accuracy: 0.0350
Epoch 116/2222
37/37 [==============================] - 0s 4ms/step - loss: 0.0121 - accuracy: 0.0345 - val_loss: 2.2704 - val_accuracy: 0.0350
Epoch 117/2222
37/37 [==============================] - 0s 4ms/step - loss: 0.0121 - accuracy: 0.0345 - val_loss: 2.2703 - val_accuracy: 0.0350
Epoch 118/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0121 - accuracy: 0.0345 - val_loss: 2.2697 - val_accuracy: 0.0350
Epoch 119/2222
37/37 [==============================] - 0s 4ms/step - loss: 0.0121 - accuracy: 0.0345 - val_loss: 2.2697 - val_accuracy: 0.0350
Epoch 120/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0121 - accuracy: 0.0345 - val_loss: 2.2693 - val_accuracy: 0.0350
Epoch 121/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0121 - accuracy: 0.0345 - val_loss: 2.2688 - val_accuracy: 0.0350
Epoch 1

Epoch 172/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0116 - accuracy: 0.0345 - val_loss: 2.2342 - val_accuracy: 0.0350
Epoch 173/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0116 - accuracy: 0.0345 - val_loss: 2.2337 - val_accuracy: 0.0350
Epoch 174/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0116 - accuracy: 0.0345 - val_loss: 2.2325 - val_accuracy: 0.0350
Epoch 175/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0118 - accuracy: 0.0345 - val_loss: 2.2315 - val_accuracy: 0.0350
Epoch 176/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0119 - accuracy: 0.0345 - val_loss: 2.2307 - val_accuracy: 0.0350
Epoch 177/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0117 - accuracy: 0.0345 - val_loss: 2.2293 - val_accuracy: 0.0350
Epoch 178/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0117 - accuracy: 0.0345 - val_loss: 2.2280 - val_accuracy:

Epoch 229/2222
37/37 [==============================] - 0s 4ms/step - loss: 0.0113 - accuracy: 0.0345 - val_loss: 2.1492 - val_accuracy: 0.0350
Epoch 230/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0114 - accuracy: 0.0345 - val_loss: 2.1464 - val_accuracy: 0.0350
Epoch 231/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0115 - accuracy: 0.0345 - val_loss: 2.1456 - val_accuracy: 0.0350
Epoch 232/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0115 - accuracy: 0.0345 - val_loss: 2.1429 - val_accuracy: 0.0350
Epoch 233/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0114 - accuracy: 0.0345 - val_loss: 2.1399 - val_accuracy: 0.0350
Epoch 234/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0115 - accuracy: 0.0345 - val_loss: 2.1403 - val_accuracy: 0.0350
Epoch 235/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0115 - accuracy: 0.0345 - val_loss: 2.1353 - val_accuracy:

...

37/37 [==============================] - 0s 5ms/step - loss: 0.0090 - accuracy: 0.0345 - val_loss: 1.0360 - val_accuracy: 0.0350
Epoch 1980/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0090 - accuracy: 0.0345 - val_loss: 1.0362 - val_accuracy: 0.0350
Epoch 1981/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0090 - accuracy: 0.0345 - val_loss: 1.0357 - val_accuracy: 0.0350
Epoch 1982/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0090 - accuracy: 0.0345 - val_loss: 1.0361 - val_accuracy: 0.0350
Epoch 1983/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0090 - accuracy: 0.0345 - val_loss: 1.0351 - val_accuracy: 0.0350
Epoch 1984/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0091 - accuracy: 0.0345 - val_loss: 1.0361 - val_accuracy: 0.0350
Epoch 1985/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0090 - accuracy: 0.0345 - val_loss: 1.0355 - val_accuracy: 0.0350
E

37/37 [==============================] - 0s 4ms/step - loss: 0.0091 - accuracy: 0.0345 - val_loss: 1.0316 - val_accuracy: 0.0350
Epoch 2036/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0091 - accuracy: 0.0345 - val_loss: 1.0324 - val_accuracy: 0.0350
Epoch 2037/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0090 - accuracy: 0.0345 - val_loss: 1.0322 - val_accuracy: 0.0350
Epoch 2038/2222
37/37 [==============================] - 0s 4ms/step - loss: 0.0090 - accuracy: 0.0345 - val_loss: 1.0324 - val_accuracy: 0.0350
Epoch 2039/2222
37/37 [==============================] - 0s 4ms/step - loss: 0.0090 - accuracy: 0.0345 - val_loss: 1.0312 - val_accuracy: 0.0350
Epoch 2040/2222
37/37 [==============================] - 0s 4ms/step - loss: 0.0090 - accuracy: 0.0345 - val_loss: 1.0323 - val_accuracy: 0.0350
Epoch 2041/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0090 - accuracy: 0.0345 - val_loss: 1.0312 - val_accuracy: 0.0350
E

37/37 [==============================] - 0s 4ms/step - loss: 0.0092 - accuracy: 0.0345 - val_loss: 1.0290 - val_accuracy: 0.0350
Epoch 2092/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0091 - accuracy: 0.0345 - val_loss: 1.0289 - val_accuracy: 0.0350
Epoch 2093/2222
37/37 [==============================] - 0s 4ms/step - loss: 0.0090 - accuracy: 0.0345 - val_loss: 1.0287 - val_accuracy: 0.0350
Epoch 2094/2222
37/37 [==============================] - 0s 4ms/step - loss: 0.0091 - accuracy: 0.0345 - val_loss: 1.0293 - val_accuracy: 0.0350
Epoch 2095/2222
37/37 [==============================] - 0s 4ms/step - loss: 0.0090 - accuracy: 0.0345 - val_loss: 1.0286 - val_accuracy: 0.0350
Epoch 2096/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0090 - accuracy: 0.0345 - val_loss: 1.0291 - val_accuracy: 0.0350
Epoch 2097/2222
37/37 [==============================] - 0s 4ms/step - loss: 0.0090 - accuracy: 0.0345 - val_loss: 1.0290 - val_accuracy: 0.0350
E

37/37 [==============================] - 0s 4ms/step - loss: 0.0091 - accuracy: 0.0345 - val_loss: 1.0265 - val_accuracy: 0.0350
Epoch 2148/2222
37/37 [==============================] - 0s 4ms/step - loss: 0.0091 - accuracy: 0.0345 - val_loss: 1.0254 - val_accuracy: 0.0350
Epoch 2149/2222
37/37 [==============================] - 0s 4ms/step - loss: 0.0090 - accuracy: 0.0345 - val_loss: 1.0262 - val_accuracy: 0.0350
Epoch 2150/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0090 - accuracy: 0.0345 - val_loss: 1.0257 - val_accuracy: 0.0350
Epoch 2151/2222
37/37 [==============================] - 0s 4ms/step - loss: 0.0090 - accuracy: 0.0345 - val_loss: 1.0262 - val_accuracy: 0.0350
Epoch 2152/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0090 - accuracy: 0.0345 - val_loss: 1.0258 - val_accuracy: 0.0350
Epoch 2153/2222
37/37 [==============================] - 0s 4ms/step - loss: 0.0090 - accuracy: 0.0345 - val_loss: 1.0267 - val_accuracy: 0.0350
E

37/37 [==============================] - 0s 5ms/step - loss: 0.0091 - accuracy: 0.0345 - val_loss: 1.0235 - val_accuracy: 0.0350
Epoch 2204/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0090 - accuracy: 0.0345 - val_loss: 1.0239 - val_accuracy: 0.0350
Epoch 2205/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0090 - accuracy: 0.0345 - val_loss: 1.0234 - val_accuracy: 0.0350
Epoch 2206/2222
37/37 [==============================] - 0s 4ms/step - loss: 0.0090 - accuracy: 0.0345 - val_loss: 1.0243 - val_accuracy: 0.0350
Epoch 2207/2222
37/37 [==============================] - 0s 4ms/step - loss: 0.0089 - accuracy: 0.0345 - val_loss: 1.0233 - val_accuracy: 0.0350
Epoch 2208/2222
37/37 [==============================] - 0s 5ms/step - loss: 0.0090 - accuracy: 0.0345 - val_loss: 1.0239 - val_accuracy: 0.0350
Epoch 2209/2222
37/37 [==============================] - 0s 4ms/step - loss: 0.0089 - accuracy: 0.0345 - val_loss: 1.0231 - val_accuracy: 0.0350
E

In [55]:
from keras.layers import Concatenate, Dense, Dropout

def RecommenderNet(n_users, n_movies, n_factors, min_rating, max_rating):

    user = Input(shape=(1,))
    u = EmbeddingLayer(n_users, n_factors)(user)

    movie = Input(shape=(1,))
    m = EmbeddingLayer(n_movies, n_factors)(movie)

    x = Concatenate()([u, m])
    x = Dropout(0.05)(x)

    x = Dense(14, kernel_initializer='he_normal')(x)
    x = Activation('relu')(x)
    x = Dropout(0.3)(x)

    x = Dense(1, kernel_initializer='he_normal')(x)
    x = Activation('sigmoid')(x)
    x = Lambda(lambda x: x * (max_rating - min_rating) + min_rating)(x)    

    model = Model(inputs=[user, movie], outputs=x)
    opt = Adam(lr=0.0007)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model



In [56]:
model = RecommenderNet(n_users, n_movies, n_factors, min_rating, max_rating)
model.summary()
history = model.fit(x=X_train_array, y=y_train, batch_size=128, epochs=8,
                    verbose=1, validation_data=(X_test_array, y_test))

Model: "model_27"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_55 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 input_56 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 embedding_56 (Embedding)       (None, 1, 60)        184560      ['input_55[0][0]']               
                                                                                                  
 embedding_57 (Embedding)       (None, 1, 60)        59880       ['input_56[0][0]']               
                                                                                           

In [57]:
## save model 2 ##
model.save("./cf_ffn_8ep.h5")


In [177]:
from functools import reduce

user_enc.transform([2])

def l2_norm(vect):
    """ l2 norm """
    add = lambda x, y: x + y
    tmp = [x*x for x in vect.tolist()]
    res = np.sqrt(reduce(add, tmp))
    return res

def cosine_sim(v1, v2, eps1=1e-4, eps2=1e-4):
    """ cosine similarity """
    div = (l2_norm(v1) + eps1) * (l2_norm(v2) + eps2)
    res = np.dot(v1, v2) / div
    return res
    

def find_n_closest(id_, ids, user_encoder, embed, n=100):
    """ find n closest vectors """
    minus_one = copy(ids)
    minus_one.remove(id_)
    k = user_encoder.transform([id_])[0]
    encoded_ids = user_encoder.transform(minus_one)
    vect = embed[0][k][...]
    res = [(i, cosine_sim(vect, embed[0][encoded_ids[i]][...])) for i in range(len(minus_one))]
    sort = sorted(res, key=lambda x: x[1], reverse=True)[:n]
    similars = [elem[1] for elem in sort]
    actuals = [minus_one[elem[0]] for elem in sort]
    return actuals, similars



Вычисление оценки рейтинга по схожим клиентам (User based Collaborative Filtering)

find_rating_for_dish_i = Rui + SUM(S(u,v) (Rvi - Rv_avg)) / SUM(S(u,v))

S(u,v) - функция сходства клиентов (u,v) - cosine simmilarity

In [194]:
def load_matrix(ratings, user_encoder, recipe_encoder):
    """ Construct matrix """
    X = ratings[['user', 'recipe']].values
    y = ratings['rating'].values
    print(X.shape, y.shape)
    n_users = ratings['user'].nunique()
    n_movies = ratings['recipe'].nunique()

    mat = sp.dok_matrix((n_users + 1, n_movies + 1), dtype=np.float32)
    for i in range(len(y)):
        if (y[i] > 0):
            mat[X[i][0], X[i][1]] = y[i]
    popular = np.zeros((n_movies))
    for i in range(len(y)):
        popular[X[i][1]] += 1
    return mat, popular



def average_rating(k, ratings):
    """ average_rating """
    vals = dict(ratings[k])
    summ = reduce(lambda x,y: x + y, vals.values()) / ratings.shape[0]
    return summ


def find_rating_for_dish_i(user, i, closest, similars, matrix, user_enc, item_enc):
    """ Вычисление оценки рейтинга по схожим клиентам 
        (User based Collaborative Filtering) 
        arguments:
            user: actual user_id
            i: translated 
            """
    n = item_enc.transform([i])[0]
    avg_u = average_rating(user_enc.transform([user])[0], matrix)
    weighted_sum = 0.
    summ = 0.
    for k in range(len(closest)):
        j = user_enc.transform([closest[k]])[0]
        if matrix[j, n] > 0:
            rate = matrix[j, n]
            avg_v = average_rating(j, matrix)
            weighted_sum += similars[k] * (rate - avg_v)
            summ += similars[k]
    if summ > 0:
        return avg_u + (weighted_sum/summ)
    return 1


mm, popular = load_matrix(ratings, user_enc, item_enc)
act_user = 2
closest_users, similarities = find_n_closest(act_user, list(actuals), user_enc, uembed, n=100)

rating_list = [find_rating_for_dish_i(2, i, closest_users, similarities, mm, user_enc, item_enc) for i in actual_items]


(5150, 2) (5150,)


Найдем самые популярные рейтинги и блюда


In [195]:
print(rating_list, popular)
print(len(rating_list), len(popular))

[1, 2.000649983750406, 1, 1.0006499837504061, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4.011374715632109, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2.000649983750406, 4.000649983750407, 5.000649983750406, 1, 4.000649983750407, 1.0006499837504061, 5.000649983750406, 3.0006499837504066, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4.00891273037329, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5.000324991875203, 1, 1, 1, 1, 5.008822531467552, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5.00942476438089, 1, 1, 1, 1, 1, 1, 1, 4.0097497562560935, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5.000649983750406, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5.000649983750406, 1, 1, 1, 1, 1, 1, 4.004874878128047, 4.001299967500812, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4.0087747806304845, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5.007149821254468, 1, 1, 1, 1, 5.007799805004875, 1, 5.006764093203874, 1, 1, 4.00097497562